# Run behavior cloning

Run a behavior cloning based robot controller on a real robot. 

__TODO as of June 1, 2025__ 
* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 
